# 📈 FinLearner - Complete Demo Notebook

This notebook demonstrates all modules and functions in the **FinLearner** library.

## Table of Contents
1. [Data Loading](#1-data-loading)
2. [Technical Indicators](#2-technical-indicators)
3. [Deep Learning Models](#3-deep-learning-models)
4. [Gradient Boosting](#4-gradient-boosting)
5. [Anomaly Detection](#5-anomaly-detection)
6. [Risk Metrics](#6-risk-metrics)
7. [Portfolio Optimization](#7-portfolio-optimization)
8. [Visualization](#8-visualization)

In [ ]:
# Install dependencies if needed
# !pip install finlearner yfinance tensorflow xgboost lightgbm plotly

In [ ]:
# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import io

# Fix encoding for Windows terminals
sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')

# Import all FinLearner modules
from finlearner import (
    # Data
    DataLoader,
    # Technical
    TechnicalIndicators,
    # Deep Learning
    TimeSeriesPredictor,
    GRUPredictor,
    CNNLSTMPredictor,
    TransformerPredictor,
    EnsemblePredictor,
    # Machine Learning
    GradientBoostPredictor,
    # Anomaly
    VAEAnomalyDetector,
    # Risk
    RiskMetrics,
    historical_var,
    parametric_var,
    monte_carlo_var,
    cvar,
    max_drawdown,
    # Portfolio
    PortfolioOptimizer,
    BlackLittermanOptimizer,
    RiskParityOptimizer,
    # Visualization
    Plotter
)

print('✅ All modules imported successfully!')

---
## 1. Data Loading

The `DataLoader` class downloads historical stock data from Yahoo Finance.

In [ ]:
# Download stock data
ticker = 'AAPL'
df = DataLoader.download_data(ticker, start='2023-01-01', end='2024-01-01')

print(f'Downloaded {len(df)} days of {ticker} data')
df.head()

In [ ]:
# Quick price plot
plt.figure(figsize=(12, 4))
plt.plot(df['Close'], label=ticker)
plt.title(f'{ticker} Closing Prices')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

---
## 2. Technical Indicators

The `TechnicalIndicators` class adds various technical analysis indicators to the data.

In [ ]:
# Add all technical indicators
ti = TechnicalIndicators(df.copy())
df_with_indicators = ti.add_all()

print('Added indicators:', df_with_indicators.columns.tolist())
df_with_indicators.tail()

In [ ]:
# Plot price with moving averages
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Price + MAs
axes[0].plot(df_with_indicators['Close'], label='Close', alpha=0.8)
# Note: Column names depend on indicator implementation (e.g., MA20 or SMA_20)
# Checking typical names from library
if 'MA20' in df_with_indicators.columns:
    axes[0].plot(df_with_indicators['MA20'], label='SMA 20', linestyle='--')
elif 'SMA_20' in df_with_indicators.columns:
    axes[0].plot(df_with_indicators['SMA_20'], label='SMA 20', linestyle='--')
    
if 'EMA12' in df_with_indicators.columns:
    axes[0].plot(df_with_indicators['EMA12'], label='EMA 12', linestyle='--')
elif 'EMA_12' in df_with_indicators.columns:
    axes[0].plot(df_with_indicators['EMA_12'], label='EMA 12', linestyle='--')

axes[0].set_title('Price with Moving Averages')
axes[0].legend()

# RSI
axes[1].plot(df_with_indicators['RSI'], label='RSI', color='purple')
axes[1].axhline(70, color='red', linestyle='--', alpha=0.5)
axes[1].axhline(30, color='green', linestyle='--', alpha=0.5)
axes[1].set_title('RSI Indicator')
axes[1].legend()

plt.tight_layout()
plt.show()

---
## 3. Deep Learning Models

FinLearner provides multiple deep learning architectures for time series prediction.

### 3.1 LSTM Predictor (TimeSeriesPredictor)

In [ ]:
# LSTM Model
lstm = TimeSeriesPredictor(lookback_days=30)
lstm.fit(df, epochs=5)

lstm_predictions = lstm.predict(df)
print(f'LSTM predictions shape: {lstm_predictions.shape}')

### 3.2 GRU Predictor

In [ ]:
# GRU Model
gru = GRUPredictor(lookback_days=30, gru_units=(64, 32))
gru.fit(df, epochs=5)

gru_predictions = gru.predict(df)
print(f'GRU predictions shape: {gru_predictions.shape}')

### 3.3 CNN-LSTM Hybrid

In [ ]:
# CNN-LSTM Hybrid Model
cnn_lstm = CNNLSTMPredictor(lookback_days=30, filters=32, kernel_size=3)
cnn_lstm.fit(df, epochs=5)

cnn_lstm_predictions = cnn_lstm.predict(df)
print(f'CNN-LSTM predictions shape: {cnn_lstm_predictions.shape}')

### 3.4 Transformer Predictor

In [ ]:
# Transformer Model with self-attention
transformer = TransformerPredictor(lookback_days=30, num_heads=4, d_model=64)
transformer.fit(df, epochs=5)

transformer_predictions = transformer.predict(df)
print(f'Transformer predictions shape: {transformer_predictions.shape}')

### 3.5 Ensemble Predictor

In [ ]:
# Ensemble: Weighted combination of LSTM + GRU + Attention
ensemble = EnsemblePredictor(
    lookback_days=30, 
    weights={'lstm': 0.4, 'gru': 0.3, 'attention': 0.3}
)
ensemble.fit(df, epochs=5)

ensemble_predictions = ensemble.predict(df)
print(f'Ensemble predictions shape: {ensemble_predictions.shape}')

In [ ]:
# Formatting helper
def fmt_mae(pred, actual):
    # Align lengths by taking the last N points
    p = pred[-len(actual):].flatten()
    mae = np.mean(np.abs(p - actual))
    return f'{mae:.2f}'

# Compare all model predictions
fig, ax = plt.subplots(figsize=(14, 6))

# 1. Determine minimum length among all predictions
min_len = min(len(lstm_predictions), len(gru_predictions), 
              len(cnn_lstm_predictions), len(transformer_predictions), 
              len(ensemble_predictions))

# 2. Get actual prices aligned to that length
actual = df['Close'].values[-min_len:]

# 3. Print metrics
print('\nMean Absolute Error:')
print(f'  LSTM:        {fmt_mae(lstm_predictions, actual)}')
print(f'  GRU:         {fmt_mae(gru_predictions, actual)}')
print(f'  CNN-LSTM:    {fmt_mae(cnn_lstm_predictions, actual)}')
print(f'  Transformer: {fmt_mae(transformer_predictions, actual)}')
print(f'  Ensemble:    {fmt_mae(ensemble_predictions, actual)}')

# 4. Plot aligned predictions
ax.plot(actual, label='Actual', linewidth=2, color='black')
ax.plot(lstm_predictions[-len(actual):].flatten(), label='LSTM', alpha=0.7)
ax.plot(gru_predictions[-len(actual):].flatten(), label='GRU', alpha=0.7)
ax.plot(cnn_lstm_predictions[-len(actual):].flatten(), label='CNN-LSTM', alpha=0.7)
ax.plot(transformer_predictions[-len(actual):].flatten(), label='Transformer', alpha=0.7)
ax.plot(ensemble_predictions[-len(actual):].flatten(), label='Ensemble', alpha=0.7)

ax.set_title('Deep Learning Model Comparison')
ax.set_xlabel('Time')
ax.set_ylabel('Price')
ax.legend()
plt.show()

---
## 4. Gradient Boosting

The `GradientBoostPredictor` supports both XGBoost and LightGBM with automatic feature engineering.

In [ ]:
# XGBoost Predictor
xgb_predictor = GradientBoostPredictor(
    backend='xgboost',
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1
)

xgb_predictor.fit(df)
xgb_predictions = xgb_predictor.predict(df)

print(f'XGBoost predictions shape: {xgb_predictions.shape}')

In [ ]:
# Feature importance
importance = xgb_predictor.feature_importance()

# Plot top 10 features
top_10 = importance.head(10)
plt.figure(figsize=(10, 6))
plt.barh(top_10['feature'], top_10['importance'], color='steelblue')
plt.xlabel('Importance')
plt.title('Top 10 Feature Importances (XGBoost)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# LightGBM Predictor
lgb_predictor = GradientBoostPredictor(
    backend='lightgbm',
    n_estimators=100,
    max_depth=6
)

lgb_predictor.fit(df)
lgb_predictions = lgb_predictor.predict(df)

print(f'LightGBM predictions shape: {lgb_predictions.shape}')

---
## 5. Anomaly Detection

The `VAEAnomalyDetector` uses a Variational Autoencoder to identify unusual price patterns.

In [ ]:
# VAE Anomaly Detector
vae = VAEAnomalyDetector(
    lookback_days=20,
    latent_dim=8,
    hidden_dims=(64, 32)
)

vae.fit(df, epochs=20, batch_size=16)

In [ ]:
# Detect anomalies
anomaly_scores = vae.detect_anomalies(df)
anomaly_df = vae.get_anomalies(df, percentile=95)

print(f'Total anomalies detected: {anomaly_df["Is_Anomaly"].sum()}')
anomaly_df[anomaly_df['Is_Anomaly']].head(10)

In [ ]:
# Plot anomalies
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Price with anomaly markers
axes[0].plot(anomaly_df['Close'], label='Price', color='steelblue')
anomalies = anomaly_df[anomaly_df['Is_Anomaly']]
axes[0].scatter(anomalies.index, anomalies['Close'], color='red', s=50, label='Anomaly', zorder=5)
axes[0].set_title('Price with Detected Anomalies')
axes[0].legend()

# Anomaly scores
axes[1].plot(anomaly_df.index, anomaly_df['Anomaly_Score'], color='orange')
axes[1].axhline(vae.reconstruction_threshold, color='red', linestyle='--', label='Threshold')
axes[1].set_title('Anomaly Scores')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Get latent space representation
latent = vae.get_latent_representation(df)
print(f'Latent representation shape: {latent.shape}')

# Visualize first 2 dimensions
plt.figure(figsize=(8, 6))
plt.scatter(latent[:, 0], latent[:, 1], c=range(len(latent)), cmap='viridis', alpha=0.6)
plt.colorbar(label='Time Index')
plt.xlabel('Latent Dim 1')
plt.ylabel('Latent Dim 2')
plt.title('VAE Latent Space Representation')
plt.show()

---
## 6. Risk Metrics

The `RiskMetrics` class provides comprehensive risk analysis including VaR, CVaR, and drawdown metrics.

In [ ]:
# Create RiskMetrics from prices
risk = RiskMetrics.from_prices(df['Close'])

# Get risk summary
summary = risk.summary(confidence=0.95)
print('\n📊 Risk Summary (95% confidence):')
summary

### 6.1 Value at Risk (VaR)

In [ ]:
# Different VaR calculation methods
returns = df['Close'].pct_change().dropna()

var_historical = risk.historical_var(confidence=0.95)
var_parametric = risk.parametric_var(confidence=0.95)
var_monte_carlo = risk.monte_carlo_var(confidence=0.95, simulations=10000)
var_cornish_fisher = risk.cornish_fisher_var(confidence=0.95)

print('Value at Risk (95% confidence):')
print(f'  Historical:     {var_historical:.4f} ({var_historical*100:.2f}%)')
print(f'  Parametric:     {var_parametric:.4f} ({var_parametric*100:.2f}%)')
print(f'  Monte Carlo:    {var_monte_carlo:.4f} ({var_monte_carlo*100:.2f}%)')
print(f'  Cornish-Fisher: {var_cornish_fisher:.4f} ({var_cornish_fisher*100:.2f}%)')

In [ ]:
# Standalone VaR functions
print('\nUsing standalone functions:')
print(f'  historical_var: {historical_var(returns, 0.95):.4f}')
print(f'  parametric_var: {parametric_var(returns, 0.95):.4f}')
print(f'  monte_carlo_var: {monte_carlo_var(returns, 0.95, simulations=10000):.4f}')

### 6.2 Conditional VaR (CVaR / Expected Shortfall)

In [ ]:
# CVaR - Expected loss beyond VaR
cvar_hist = risk.cvar(confidence=0.95, method='historical')
cvar_param = risk.cvar(confidence=0.95, method='parametric')

print('Conditional VaR (95% confidence):')
print(f'  Historical CVaR: {cvar_hist:.4f} ({cvar_hist*100:.2f}%)')
print(f'  Parametric CVaR: {cvar_param:.4f} ({cvar_param*100:.2f}%)')
print(f'\nNote: CVaR > VaR because it measures tail risk beyond VaR')

### 6.3 Maximum Drawdown

In [ ]:
# Maximum Drawdown analysis
max_dd = risk.max_drawdown()
dd_series = risk.max_drawdown(as_series=True)
calmar = risk.calmar_ratio()

print('Drawdown Metrics:')
print(f'  Max Drawdown: {max_dd:.4f} ({max_dd*100:.2f}%)')
print(f'  Calmar Ratio: {calmar:.4f}')

In [ ]:
# Visualize drawdown over time
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Cumulative returns
cumulative = (1 + returns).cumprod()
axes[0].plot(cumulative, color='steelblue')
axes[0].set_title('Cumulative Returns')
axes[0].set_ylabel('Growth Factor')

# Drawdown
axes[1].fill_between(dd_series.index, dd_series.values, 0, color='red', alpha=0.3)
axes[1].plot(dd_series, color='red')
axes[1].set_title('Drawdown Over Time')
axes[1].set_ylabel('Drawdown')

plt.tight_layout()
plt.show()

---
## 7. Portfolio Optimization

FinLearner provides three portfolio optimization strategies.

### 7.1 Markowitz Mean-Variance Optimization

In [ ]:
# Portfolio of tech stocks
tickers = ['AAPL', 'GOOG', 'MSFT', 'AMZN', 'META']

# Markowitz Optimizer
markowitz = PortfolioOptimizer(
    tickers=tickers,
    start='2023-01-01',
    end='2024-01-01'
)

In [ ]:
# Optimize portfolio
results, allocations, metrics = markowitz.optimize(num_portfolios=1000)

print('📊 Markowitz Optimal Portfolio:')
print(allocations)

In [ ]:
# Efficient Frontier
markowitz.plot_efficient_frontier()

### 7.2 Black-Litterman Optimization

In [ ]:
# Black-Litterman with investor views
bl_optimizer = BlackLittermanOptimizer(
    tickers=tickers,
    start='2023-01-01',
    end='2024-01-01'
)

# Define views: AAPL will return 20%, MSFT 15%
views = {
    'AAPL': 0.20,
    'MSFT': 0.15
}

bl_allocation, bl_metrics = bl_optimizer.optimize(views=views)

print('📊 Black-Litterman Optimal Portfolio:')
print(bl_allocation)
print(f'\nExpected Return: {bl_metrics["expected_return"]:.2%}')
print(f'Volatility: {bl_metrics["volatility"]:.2%}')
print(f'Sharpe Ratio: {bl_metrics["sharpe_ratio"]:.2f}')

### 7.3 Risk Parity Optimization

In [ ]:
# Risk Parity - Equal risk contribution
rp_optimizer = RiskParityOptimizer(
    tickers=tickers,
    start='2023-01-01',
    end='2024-01-01'
)

rp_allocation, rp_metrics = rp_optimizer.optimize()

print('📊 Risk Parity Portfolio:')
print(rp_allocation)
print('\n📈 Metrics:')
print(rp_metrics)

In [ ]:
# Compare allocation strategies
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Markowitz
axes[0].pie(allocations['allocation'], labels=allocations.index, autopct='%1.1f%%')
axes[0].set_title('Markowitz')

# Black-Litterman
axes[1].pie(bl_allocation['Weight'], labels=bl_allocation.index, autopct='%1.1f%%')
axes[1].set_title('Black-Litterman')

# Risk Parity
axes[2].pie(rp_allocation['Weight'], labels=rp_allocation.index, autopct='%1.1f%%')
axes[2].set_title('Risk Parity')

plt.suptitle('Portfolio Allocation Comparison', fontsize=14)
plt.tight_layout()
plt.show()

---
## 8. Visualization

The `Plotter` class provides interactive candlestick charts with indicators.

In [ ]:
# Interactive candlestick chart
fig = Plotter.candlestick(df, title=f'{ticker} Candlestick Chart')
fig.show()

In [ ]:
# Candlestick with volume
fig = Plotter.candlestick(df_with_indicators, title=f'{ticker} with Volume', show_volume=True)
fig.show()

---
## 📚 Summary

This notebook demonstrated all major features of FinLearner:

| Module | Features |
|--------|----------|
| **Data** | `DataLoader.download_data()` |
| **Technical** | `TechnicalIndicators.add_all()` - RSI, MACD, Bollinger, etc. |
| **Deep Learning** | LSTM, GRU, CNN-LSTM, Transformer, Ensemble |
| **ML** | XGBoost, LightGBM with feature engineering |
| **Anomaly** | VAE-based anomaly detection |
| **Risk** | VaR (4 methods), CVaR, Max Drawdown, Calmar |
| **Portfolio** | Markowitz, Black-Litterman, Risk Parity |
| **Visualization** | Interactive Plotly charts |